# TAP: Table Access Protocol Services

Table Access Protocol Services allow more direct and flexible access to astronomical data than the simpler types of IVOA standard data services. Queries are built with a SQL-like language ADQL (Astronomical Data Query Language), and can include geographic / spatial queries as well as filtering on other characteristics of the data. This also allows the user fine-grained control over the returned columns, unlike the fixed set from cone, image, and spectral services.

For this example, we're going to find GALEX images in MAST using a TAP service on the data holdings in the Common Archive Observation Model. The data model itself is a VO standard, so data from various archives around the world can be cross-correlated by querying VO services that provide it. 

In [3]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display
from html import unescape

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

# Use the NASA_NAVO/astroquery
from navo_utils.registry import Registry

# Use the astroquery TapPlus library.
from astroquery.utils.tap.core import TapPlus

# Finding TAP services

To work with table services one can begin by searching for them in any registry and filtering on the results, just as we have done for other service types. In fact, the registry interface is itself a TAP service, for which we've built helper functions for common useful queries.



In [4]:
tap_services_CAOM=Registry.query(keyword='caom',service_type='table', publisher='Space Telescope')
print('Found {} results:'.format(len(tap_services_CAOM)))
tap_url = unescape(tap_services_CAOM[0]['access_url'])
print(tap_url) 


Found 1 results:
http://vao.stsci.edu/CAOMTAP/TapService.aspx


Upon finding the TAP service, one can query the individual service this with the astroquery utility TapPlus. TapPlus was created as a layer in the GAIA archive utilities, but works for all TAP services. Documentation at: https://github.com/astropy/astroquery/blob/master/docs/utils/tap.rst

In general, one opens a connection to the service URL. Then, if one does not already know the database table information associated with the service, one can ask the service for it. Since we're querying a standard service using the known CAOM and ObsCore data models, we know this information from http://www.ivoa.net/documents/ObsCore/. 
Then one can build and run the main query, either synchronously or asynchronously. We'll do a synchronous call. 

This first call is the equivalent of a cone search:

In [5]:
CAOM_service = TapPlus(url=tap_url)

job = CAOM_service.launch_job("""
    SELECT * FROM ivoa.Obscore 
    WHERE CONTAINS(POINT('ICRS', 16.0, 40.0),s_region)=1
  """)
CAOM_results = job.get_results()
print(CAOM_results)

Created TAP+ (v1.0.1) - Connection:
	Host: vao.stsci.edu
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Launched query: '
    SELECT  TOP 2000 * FROM ivoa.Obscore 
    WHERE CONTAINS(POINT('ICRS', 16.0, 40.0),s_region)=1
  '
Retrieving sync. results...
Query finished.
dataproduct_type calib_level obs_collection ... facility_name instrument_name
---------------- ----------- -------------- ... ------------- ---------------
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2          GALEX ...       CALTECH           GALEX
           image           2     

Since this is TAP and not just a cone search, we can narrow down our query to, in this case, GALEX images. And we can specify only the returned column information we want.  Geometrically based queries allow one to do this equivalent of a cone search or bounded-box footprint search or several other shapes.

To only get the position, estimated file size, and actual link to data for each returned result, from the prior cone search but further filtered, one could form the ADQL as such:

In [6]:
job = CAOM_service.launch_job("""
    SELECT top 10 s_ra, s_dec, access_estsize, access_url FROM ivoa.Obscore 
    WHERE CONTAINS(POINT('ICRS', 16.0, 40.0),s_region)=1
    AND obs_collection = "GALEX" AND dataproduct_type = 'image'
  """)
CAOM_results = job.get_results()

print(CAOM_results[1])

Launched query: '
    SELECT top 10 s_ra, s_dec, access_estsize, access_url FROM ivoa.Obscore 
    WHERE CONTAINS(POINT('ICRS', 16.0, 40.0),s_region)=1
    AND obs_collection = "GALEX" AND dataproduct_type = 'image'
  '
Retrieving sync. results...
Query finished.
       s_ra              s_dec        access_estsize                                                                           access_url                                                                         
------------------ ------------------ -------------- -------------------------------------------------------------------------------------------------------------------------------------------------------------
16.356213326750701 39.772757075359799        5520978 https://mast.stsci.edu/portal/Download/file?uri=http://galex.stsci.edu/data/GR6/pipe/02-vsn/50042-AIS_42/d/01-main/0001-img/07-try/AIS_42_sg09-nd-int.fits.gz


# Appendix: Documentation on the Standards

### Table Access Protocol 
* IVOA standard for RESTful web service access to tabular data
* http://www.ivoa.net/documents/TAP/

### Common Archive Observation Model
* IVOA standard created primarily at the Canadian Astronomy Data Center
* http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/doc/caom/

### Astronomical Query Data Language (2.0)
* IVOA standard for querying astronomical data in tabular format, with geometric search support
* http://www.ivoa.net/documents/latest/ADQL.html

### TapPlus 
* 
* http://astroquery.readthedocs.io/en/latest/utils/tap.html


 